In [1]:
import pandas as pd
from tqdm import tqdm
import thulac
import re
import numpy as np
import json
import tensorflow.keras as keras

In [2]:
comments = np.load('data/comments.npy')
stars = np.load('data/stars.npy')
# 测试上一步中整合后的数据集
print(sum(stars == 1))
print(sum(stars == 2))
print(sum(stars == 3))
print(sum(stars == 4))
print(sum(stars == 5))

15000
15000
10000
10000
10000


In [3]:
# 将 1~2 星的影评归类为编号 0
# 将 3~5 星的影评归类为编号 1
stars[stars <= 2] = 0
stars[stars >= 3] = 1
print(sum(stars == 0))
print(sum(stars == 1))

30000
30000


In [4]:
# 使用 thulac 作为分词工具
# user_dict: 设置用户词典，用户词典中的词会被打上 uw 标签。词典中每一个词一行，UTF8编码
# T2S: 默认False, 是否将句子从繁体转化为简体
# seg_only: 默认False, 时候只进行分词，不进行词性标注
# filt: 默认False, 是否使用过滤器去除一些没有意义的词语，例如 “可以”。
# model_path: 设置模型文件所在文件夹，默认为 models/
thu = thulac.thulac(seg_only=True, T2S=True, filt=True)
word_set = set()#使用集合:防止分词后出现重复词语
# 遍历影评集合，使用 thu.cut 执行文本分词
for i in tqdm(range(len(comments))):
    comment = comments[i]

    text = thu.cut(comment, text=True)#进行文本分词
    text = text.split(' ')#以空格为分隔符，将文本转换为单词数组
    word_set.update(text)#将单词数组合并到单词集合中，自动删除重复的单词

  0%|          | 40/60000 [00:00<02:32, 393.14it/s]

Model loaded succeed


100%|██████████| 60000/60000 [01:05<00:00, 911.19it/s] 


In [5]:
# 单词集合转换为 NumpyArray 类型的单词数组
word_numpy = np.array(list(word_set))
#存储数据
np.save('data/wordDict.npy', word_numpy)
np.save('data/labels.npy', stars)

In [6]:
#即使对句子进行了分词操作，计算机无法理解字符串数据。
#还需将字符串的单词转换为向量的单词-word2vec文本向量化
# 定义单词字典
word_dict = {
}

word_dict["<PAD>"] = 0
word_dict["<START>"] = 1
word_dict["<UNK>"] = 2
word_dict["<UNUSED>"] = 3
#cnt为当前编号
cnt = 4

In [7]:
# 遍历单词数组并将其编号，存入单词字典
for w in word_numpy:
    word_dict[w] = cnt
    cnt += 1

In [8]:
# 为了后续使用方便，将单词数组以 JSON 文件的形式存入
fout = open('data/word_dict.json', 'w+', encoding='utf8')
json.dump(word_dict, fout)
fout.close()

In [9]:
comments = np.load('data/comments.npy')

fin = open('data/word_dict.json', 'r+', encoding='utf8')
word_dict = json.load(fin)
fin.close()

In [10]:
# 将 filt 参数设置为 false
thu = thulac.thulac(seg_only=True, T2S=True, filt=False)
def wordEmbedding(comment):
    global thu,word_dict

    texts = thu.cut(comment, text=True)
    texts = texts.split(' ')
    
    encodedReview = np.array([1]) #  创建句子向量数组，预先填入 <START> 标签
    
    for text in texts:
        if(word_dict.get(text) != None):
            encodedReview = np.append(encodedReview, word_dict[text])
        else:
            encodedReview = np.append(encodedReview, 2) # <UNK>

    return encodedReview

Model loaded succeed


In [11]:
data = []

In [12]:
for i in tqdm(range(comments.shape[0])):
    comment = comments[i]
    encodedComment = wordEmbedding(comment)
    data.append(encodedComment)

100%|██████████| 60000/60000 [01:06<00:00, 909.03it/s] 


In [13]:
data = np.array(data)

In [14]:
# 不同的影评中的词语数不同,将数据长度进行统一
# 使用 keras.preprocessing.sequence.pad_sequences 函数统一影评数据长度
dataPaddinged = keras.preprocessing.sequence.pad_sequences(data,
                                                        value=word_dict['<PAD>'],
                                                        padding='post',
                                                        maxlen=200)

In [15]:
np.save('data/data.npy', dataPaddinged)